In [1]:
import torch
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding

2023-06-12 21:50:18.377432: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 21:50:19.587921: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/cudnn/cuda-9.1/7.1/cuda/lib64:/opt/cuda/11.1.1/extras/CUPTI/lib64/:/opt/cudnn/cuda-9.1/7.1/cuda/lib64:/opt/cuda/11.1.1/extras/CUPTI/lib64/::/opt/cuda/11.1.1:/opt/cuda/11.1.1
2023-06-12 21:50:19.588199: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object f

In [2]:
from datasets import load_dataset
raw_dataset = load_dataset("sst2", split="validation")

Found cached dataset sst2 (/home/sachink/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


In [28]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union

from transformers.utils import PaddingStrategy

@dataclass
class DataCollatorForGenerativeClassification:
    """
    Data collator that will dynamically pad the inputs received.

    Args:
        tokenizer ([`PreTrainedTokenizer`] or [`PreTrainedTokenizerFast`]):
            The tokenizer used for encoding the data.
        padding (`bool`, `str` or [`~utils.PaddingStrategy`], *optional*, defaults to `True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:

            - `True` or `'longest'` (default): Pad to the longest sequence in the batch (or no padding if only a single
              sequence is provided).
            - `'max_length'`: Pad to a maximum length specified with the argument `max_length` or to the maximum
              acceptable input length for the model if that argument is not provided.
            - `False` or `'do_not_pad'`: No padding (i.e., can output a batch with sequences of different lengths).
        max_length (`int`, *optional*):
            Maximum length of the returned list and optionally padding length (see above).
        pad_to_multiple_of (`int`, *optional*):
            If set will pad the sequence to a multiple of the provided value.

            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
        return_tensors (`str`):
            The type of Tensor to return. Allowable values are "np", "pt" and "tf".
    """

    tokenizer: PreTrainedTokenizerBase
    label_features = None
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    return_tensors: str = "pt"


    def __call__(self, features):
        new_features = {}
        for key in features:
            if key not in ['input_ids', 'attention_mask']:
                pass
        

        batch = self.tokenizer.pad(
            features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )


        if "label" in batch:
            batch["labels"] = batch["label"]
            del batch["label"]
        if "label_ids" in batch:
            batch["labels"] = batch["label_ids"]
            del batch["label_ids"]
        return batch

In [45]:
modelname = "gpt2-large"
tokenizer = AutoTokenizer.from_pretrained(modelname)

tokenizer.pad_token = tokenizer.eos_token    
tokenizer.pad_token_id = tokenizer.eos_token_id

def get_tokenized_dataset(raw_dataset, textfield="sentence", labelfield="label", label2id=None):
        def preprocess_function(examples):
                x = tokenizer(examples[textfield], max_length=200, truncation=True)
                if label2id is not None:
                        x['labels'] = [label2id[label] for label in examples[labelfield]]
                return x

        tokenized_dataset = raw_dataset.map(preprocess_function, batched=True)
        tokenized_dataset = tokenized_dataset.remove_columns([textfield])
        tokenized_dataset.set_format("torch")

        return tokenized_dataset

tokenized_dataset = get_tokenized_dataset(raw_dataset, "sentence", "label")
# tokenized_dataset = get_tokenized_dataset(raw_dataset, "text", "label")
print("datasets and tokenizer loaded")

Loading cached processed dataset at /home/sachink/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-5ef0f9d7ba4525a8.arrow


datasets and tokenizer loaded


In [47]:
tokenized_dataset[:3]

{'idx': tensor([0, 1, 2]),
 'label': tensor([1, 0, 1]),
 'input_ids': [tensor([  270,   705,    82,   257, 23332,   290,  1690, 13891,  7002,   764,
            220]),
  tensor([  403,  2704,  8589,  4420, 30942,   290, 12111,   220]),
  tensor([47205,   514,   284,  2911,   326,   299, 16617,   318, 24357,   284,
          21030,   257,  1688,  3451,   355,   257,  5068,  1865, 47602, 26479,
            764,   220])],
 'attention_mask': [tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]}

In [11]:
model = AutoModelForCausalLM.from_pretrained(modelname)    
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.eval()
print("model loaded")

model loaded


In [7]:
alllabelstrings = [
    [  
        "This review exhibits a negative bias. ",
        "The inclination of this review is towards the negative side. ",
        "There is a unfavorable slant in this review. ",
        "The overall tone of this review is negative. ",
        "This review shows a leaning against the subject. ",
        "There is a negative inclination in this review. ",
        "The overall impression of this review is pessimistic. ",
        "This review tends to disfavor the subject being discussed. ",
        "There is a negative inclination evident in this review. ",
        "The general sentiment of this review is negative. ",
    ],
    [
        "This review exhibits a positive bias. ",
        "The inclination of this review is towards the positive side. ",
        "There is a favorable slant in this review. ",
        "The overall tone of this review is positive. ",
        "This review shows a leaning in favor of the subject. ",
        "There is a positive inclination in this review. ",
        "The overall impression of this review is optimistic. ",
        "This review tends to favor the subject being discussed. ",
        "There is a positive inclination evident in this review. ",
        "The general sentiment of this review is positive. ",
    ]
]

In [41]:
# tokenizer.pad_token = tokenizer.eos_token    
# tokenizer.pad_token_id = tokenizer.eos_token_id

num_labels = len(alllabelstrings)
num_labelstrings = len(alllabelstrings[0])

alllabelstrings_tokenized = []
for labelstrings in alllabelstrings:
    labelstrings_tokenized = []
    for labelstring in labelstrings:
        labelstrings_tokenized.append(tokenizer(labelstring, add_special_tokens=False, padding=False, return_tensors="pt").to(device))
    alllabelstrings_tokenized.append(labelstrings_tokenized)


alllabelstrings_tokenized


[[{'input_ids': tensor([[ 1212,  2423, 27508,   257,  4633, 10690,    13,   220]],
         device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')},
  {'input_ids': tensor([[  464, 36793,   286,   428,  2423,   318,  3371,   262,  4633,  1735,
              13,   220]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')},
  {'input_ids': tensor([[ 1858,   318,   257, 38206,  1017,   415,   287,   428,  2423,    13,
             220]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')},
  {'input_ids': tensor([[ 464, 4045, 8216,  286,  428, 2423,  318, 4633,   13,  220]],
         device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')},
  {'input_ids': tensor([[ 1212,  2423,  2523,   257, 21804,  1028,   262,  2426,    13,   220]],
         device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], devic

In [24]:
x = ["a", "a b", "a b c"]
tokenizer(x)

{'input_ids': [[64], [64, 275], [64, 275, 269]], 'attention_mask': [[1], [1, 1], [1, 1, 1]]}

In [48]:
def process_batch(batch, alllabelstrings_tokenized, i, j):
    labelstring_tokenized = alllabelstrings_tokenized[i][j]
    #print(batch.keys())
    batch = {k: v.to(device) for k, v in batch.items()}
    batch_size, seq_len = batch['input_ids'].size()
    label_len = labelstring_tokenized['input_ids'].size(-1)
    # print(batch_size, seq_len, label_len)
    
    expanded_batch_input_ids = batch['input_ids'].repeat_interleave(labelstring_tokenized['input_ids'].size(0), dim=0) # output size = (#labels*batch_size, L)
    expanded_label_input_ids = labelstring_tokenized['input_ids'].view(1, -1, label_len).expand(batch_size, -1, -1).contiguous().view(-1, label_len)
    input_ids = torch.cat([expanded_label_input_ids, expanded_batch_input_ids], dim=1)

    expanded_batch_attention_mask = batch['attention_mask'].repeat_interleave(labelstring_tokenized['attention_mask'].size(0), dim=0) # output size = (#labels*batch_size, L)
    expanded_label_attention_mask = labelstring_tokenized['attention_mask'].view(1, -1, label_len).expand(batch_size, -1, -1).contiguous().view(-1, label_len)
    attention_mask = torch.cat([expanded_label_attention_mask, expanded_batch_attention_mask], dim=1)
    
    label_mask = torch.ones_like(attention_mask)[:label_len] = 0.
    labels = batch['labels']
    batch['input_ids'] = input_ids
    batch['attention_mask'] = attention_mask
    bsz = input_ids.size(0)
    batch.pop("labels", None)
    batch.pop('idx', None)

    return batch, labels, batch_size, label_len, label_mask

In [14]:
def process_batch2(batch, alllabelstrings_tokenized, i):
    merged_labelstrings = alllabelstrings_tokenized[i]
    # print(merged_labelstrings)
    batch = {k: v.to(device) for k, v in batch.items()}
    batch_size, seq_len = batch['input_ids'].size()
    label_len = merged_labelstrings['input_ids'].size(-1)
    # print(batch_size, seq_len, label_len)
    
    expanded_batch_input_ids = batch['input_ids'].repeat_interleave(merged_labelstrings['input_ids'].size(0), dim=0) # output size = (#labels*batch_size, L)
    expanded_label_input_ids = merged_labelstrings['input_ids'].view(1, -1, label_len).expand(batch_size, -1, -1).contiguous().view(-1, label_len)
    input_ids = torch.cat([expanded_label_input_ids, expanded_batch_input_ids], dim=1)

    expanded_batch_attention_mask = batch['attention_mask'].repeat_interleave(merged_labelstrings['attention_mask'].size(0), dim=0) # output size = (#labels*batch_size, L)
    expanded_label_attention_mask = merged_labelstrings['attention_mask'].view(1, -1, label_len).expand(batch_size, -1, -1).contiguous().view(-1, label_len)
    attention_mask = torch.cat([expanded_label_attention_mask, expanded_batch_attention_mask], dim=1)
     
    labels = batch['labels']
    batch['input_ids'] = input_ids
    batch['attention_mask'] = attention_mask
    bsz = input_ids.size(0)
    batch.pop("labels", None)
    batch.pop('idx', None)

    return batch, labels, batch_size, label_len

In [50]:
from torch.utils.data import DataLoader

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")
data_collator.tokenizer.pad_token_id = tokenizer.eos_token_id
eval_dataloader = DataLoader(tokenized_dataset, collate_fn=data_collator, batch_size=12)

# print(tokenized_dataset['validation'])
batch = next(iter(eval_dataloader))
batch

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'idx': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]), 'input_ids': tensor([[  270,   705,    82,   257, 23332,   290,  1690, 13891,  7002,   764,
           220, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256],
        [  403,  2704,  8589,  4420, 30942,   290, 12111,   220, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256],
        [47205,   514,   284,  2911,   326,   299, 16617,   318, 24357,   284,
         21030,   257,  1688,  3451,   355,   257,  5068,  1865, 47602, 26479,
           764,   220, 50256, 50256, 50256, 50256, 50256, 50256, 5025

In [52]:
from tqdm import tqdm
import numpy as np
accurate = 0
total = 0
all_predictions = []
all_labels = []
############################################
with torch.no_grad():
    # for batch in tqdm(eval_dataloader):
    if True:
        nlls = []
        for i in range(len(alllabelstrings_tokenized)):
            perlabel_nlls = []
            for j in range(len(alllabelstrings_tokenized[i])):
                new_batch, labels, batch_size, label_len, label_mask = process_batch(batch, alllabelstrings_tokenized, i, j)
                
                outputs = model(**new_batch)
                logits1 = outputs.logits
                
                shift_logprobs = torch.nn.functional.log_softmax(logits1[..., label_len-1:-1, :], dim=-1).contiguous()
                shift_target = new_batch['input_ids'][..., label_len:].contiguous()

                nll = torch.nn.functional.nll_loss(shift_logprobs.view(-1, shift_logprobs.size(-1)), shift_target.view(-1), reduction="none", ignore_index=tokenizer.pad_token_id).view(-1, shift_target.size(-1))
                nll = nll.sum(dim=-1)/shift_target.ne(tokenizer.pad_token_id).float().sum(dim=-1)

                nll = nll.view(batch_size, 1)
                # print(nll)
                # print(nll.min(dim=1)[1].eq(batch['labels'].cuda()).int().sum().item())
                # print(nll[1])

                perlabel_nlls.append(nll)
                del new_batch
        
            nll = torch.stack(perlabel_nlls, dim=1) # bsz x label_size
            nlls.append(nll)

        nll = torch.stack(nlls, dim=2)
        nll = -torch.logsumexp(-nll, dim=-1) + np.log(num_labelstrings)

        nll = nll.min(dim=1)
        # print(nll[1])

        accurate += nll[1].eq(batch['labels'].cuda()).int().sum().item()
        total += nll[1].size(0)
        all_predictions += nll[1].tolist()
        all_labels += batch['labels'].tolist()
        # print(all_predictions)
        # print(all_labels)
        # print(accurate, "/", nll[1].size(0))


dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attention_mask', 'labels'])
dict_keys(['idx', 'input_ids', 'attentio

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_78218/2094663292.py:43 in <cell line: 8>                                          │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_78218/2094663292.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: The size of tensor a (2) must match the size of tensor b (12) at non-singleton dimension 1

In [15]:
print(f"Yelp Polarity: {accurate}/{total}, {accurate/total}")

Yelp Polarity: 31487/38000, 0.8286052631578947


In [13]:
print(f"SST-2: {accurate}/{total}, {accurate/total}")

SST-2: 31487/38000, 0.8286052631578947


In [14]:
# SST-2: 747/872, 0.856651376146789
# Yelp Polarity: 31487/38000, 0.8286052631578947
# # alllabelstrings = [
# #     [  
# #         "This review exhibits a negative bias.",
# #         "The inclination of this review is towards the negative side.",
# #         "There is a unfavorable slant in this review.",
# #         "The overall tone of this review is negative.",
# #         "This review shows a leaning against the subject.",
# #         "There is a negative inclination in this review.",
# #         "The overall impression of this review is pessimistic.",
# #         "This review tends to disfavor the subject being discussed.",
# #         "There is a negative inclination evident in this review.",
# #         "The general sentiment of this review is negative.",
# #     ],
# #     [
# #         "This review exhibits a positive bias.",
# #         "The inclination of this review is towards the positive side.",
# #         "There is a favorable slant in this review.",
# #         "The overall tone of this review is positive.",
# #         "This review shows a leaning in favor of the subject.",
# #         "There is a positive inclination in this review.",
# #         "The overall impression of this review is optimistic.",
# #         "This review tends to favor the subject being discussed.",
# #         "There is a positive inclination evident in this review.",
# #         "The general sentiment of this review is positive.",
# #     ]
# # ]

# # alllabelstrings = [["This review is leaning negative. "], ["This review is leaning positive. "]]
# # alllabelstrings = [["This review criticizes. "], ["This review appreciates. "]]

# alllabelstrings = [
#     [  
#         "This review exhibits a negative intent.",
#         #"The inclination of this review is towards the negative side.",
#         #"There is a unfavorable slant in this review.",
#         # "The overall tone of this review is negative.",
#         # "This review shows a leaning against the subject.",
#         "There is a negative inclination in this review.",
#         # "The overall impression of this review is pessimistic.",
#         # "This review tends to disfavor the subject being discussed.",
#         # "There is a negative inclination evident in this review.",
#         # "The general sentiment of this review is negative.",
#     ],
#     [
#         "This review exhibits a positive intent.",
#         #"The inclination of this review is towards the positive side.",
#         #"There is a favorable slant in this review.",
#         # "The overall tone of this review is positive.",
#         # "This review shows a leaning in favor of the subject.",
#         "There is a positive inclination in this review.",
#         # "The overall impression of this review is optimistic.",
#         # "This review tends to favor the subject being discussed.",
#         # "There is a positive inclination evident in this review.",
#         # "The general sentiment of this review is positive.",
#     ]
# ]

# num_labels = len(alllabelstrings)
# num_labelstrings = [len(labelstrings) for labelstrings in alllabelstrings]

# merged_labelstrings = []
# for labelstrings in alllabelstrings:
#     merged_labelstrings += labelstrings

# tokenizer.padding_side = "left"
# merged_labelstrings_tokenized = tokenizer(merged_labelstrings, add_special_tokens=False, padding=True, return_tensors="pt").to(device)
# tokenizer.padding_side = "right"

# print(len(merged_labelstrings))
# print(merged_labelstrings_tokenized['input_ids'].size())
# for key in merged_labelstrings_tokenized:
#     merged_labelstrings_tokenized[key] = merged_labelstrings_tokenized[key].view(num_labels, num_labelstrings[0], -1)

# tokenizer.pad_token = tokenizer.eos_token    
# tokenizer.pad_token_id = tokenizer.eos_token_id
